In [ ]:
from flask import Flask, request, jsonify
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import io

# Ініціалізація Flask додатка
app = Flask(__name__)

# Завантаження моделі
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load('games_classification_model_exp1.pth', map_location=device)
model = model.to(device)
model.eval()  # Перемикаємо модель в режим оцінки

# Трансформації для передобробки зображення
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Назви класів
class_names = ["Among Us", "Apex Legends", "Fortnite", "Forza Horizon", "Free Fire", "Genshin Impact", "God of War", "Minecraft", "Roblox",
              "Terraria"]

# Функція для передобробки зображення
def preprocess_image(image):
    image = Image.open(io.BytesIO(image)).convert("RGB")
    image = data_transforms(image)
    image = image.unsqueeze(0)  # Додаємо batch dimension
    return image.to(device)

# Ендпоінт для прийому зображень і передбачення
@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({"error": "Будь ласка, завантажте зображення"}), 400

    file = request.files['file']
    img_bytes = file.read()
    
    # Передобробка зображення
    image = preprocess_image(img_bytes)
    
   # Передбачення класу
    with torch.no_grad():
        outputs = model(image)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)  # Отримуємо ймовірності
        max_prob, predicted = torch.max(probabilities, 1)
        predicted_class = class_names[predicted.item()]    
        
        return jsonify({"result": f"This is {predicted_class} {max_prob.item()*100:.1f}%"})

# Запуск сервера
if __name__ == '__main__':
    app.run(debug=True, port=5101, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


C:\Users\bveta\AppData\Local\Temp\ipykernel_18380\3803946574.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('games_classification_model_exp1.pth', m